In [46]:
import os
import pandas as pd
import numpy as np


## ALL SCHOOOLS (based on old mapper file)

In [47]:
allschool_locations = pd.read_csv('updated_sg_zipcode_mapper_utfv2.csv', index_col = 0)
allschool_locations = allschool_locations[allschool_locations['searchval'].str.contains('SCHOOL|SKOOL|UNIVERSITY|COLLEGE')]
allschool_locations = allschool_locations[~allschool_locations['searchval'].str.contains('UNIVERSITY RD')]
allschool_locations.drop(columns=['block','street_name','address','postal'], inplace=True)
allschool_locations = allschool_locations.rename(columns={'postal.1':'postal'}).reset_index(drop = True)
allschool_locations

,lat,lng,searchval,building,postal
0,1.301061,103.781310,ACS INDEPENDENT BOARDING SCHOOL,ACS INDEPENDENT BOARDING SCHOOL,139656
1,1.319923,103.834958,ACS OLDHAM HALL BOARDING SCHOOL,ACS OLDHAM HALL BOARDING SCHOOL,309937
2,1.445891,103.802398,ADMIRALTY SECONDARY SCHOOL,ADMIRALTY SECONDARY SCHOOL,737916
3,1.379838,103.845603,ANDERSON JUNIOR COLLEGE STUDENT HOSTEL,ANDERSON JUNIOR COLLEGE STUDENT HOSTEL,567736
4,1.331181,103.941853,ANGLICAN HIGH SCHOOL,ANGLICAN HIGH SCHOOL,487012
...,...,...,...,...,...
830,1.300206,103.837317,RAFFLES COLLEGE OF HIGHER EDUCATION,111 SOMERSET,238164
831,1.275215,103.801330,SINGAPORE RAFFLES MUSIC COLLEGE,HUNDRED THOUGHTS PTE LTD,119962
832,1.357136,103.890195,YUYING SECONDARY SCHOOL,YUYING SECONDARY SCHOOL,538884
833,1.406937,103.896275,YUSOF ISHAK SECONDARY SCHOOL,YUSOF ISHAK SECONDARY SCHOOL,828676


## MOE SCHOOLS

In [94]:
moeschools = pd.read_csv('Generalinformationofmoeschools.csv')
moeschools[moeschools['school_name']=='YUSOF ISHAK SECONDARY SCHOOL']

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
339,YUSOF ISHAK SECONDARY SCHOOL,http://www.yusofishaksec.moe.edu.sg,8 SUMANG WALK,828676,65009800,na,65675502,na,yiss@moe.edu.sg,BUKIT BATOK MRT,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil


In [95]:
moeschools2 = moeschools[['school_name','address','postal_code','mrt_desc','bus_desc']]
moeschools2 = moeschools2.rename(columns={'postal_code':'postal', 'school_name':'building'})
moeschools2

,building,address,postal,mrt_desc,bus_desc
0,ADMIRALTY PRIMARY SCHOOL,11 WOODLANDS CIRCLE,738907,Admiralty Station,"TIBS 965, 964, 913"
1,ADMIRALTY SECONDARY SCHOOL,31 WOODLANDS CRESCENT,737916,ADMIRALTY MRT,904
2,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11,768643,Yishun,Yishun Ring Road - 812 (white plate); Yishun A...
3,AHMAD IBRAHIM SECONDARY SCHOOL,751 YISHUN AVENUE 7,768928,"CANBERRA MRT, YISHUN MRT","117, 167, 169, 800, 811, 812, 883, 856, 858, 8..."
4,AI TONG SCHOOL,100 Bright Hill Drive,579646,Bishan MRT,"410, 162, 52"
...,...,...,...,...,...
341,ZHANGDE PRIMARY SCHOOL,51 Jalan Membina,169485,Tiong Bahru MRT Station,"16,33,63,123,195,605,851,14,65,147,167,196,197..."
342,ZHENGHUA PRIMARY SCHOOL,9 Fajar Road,679002,LRT: Fajar Station,"190, 184, 187, 700, 700A, 921"
343,ZHENGHUA SECONDARY SCHOOL,91 SENJA ROAD,677741,JELAPANG LRT,"920, 922, 976"
344,ZHONGHUA PRIMARY SCHOOL,12 SERANGOON AVENUE 4,556095,"Ang Mo Kio MRT, Bishan MRT, Serangoon MRT","Boundary Road: 22,24,53,105,156,315,853,501,50..."


In [96]:
geocodes = pd.read_csv('cleaned_sg_zipcode_mapper_utf.csv', index_col = 0)
geocodes = geocodes[['postal','lat','lng']]
geocodes

,postal,lat,lng
0,398614,1.312763,103.883519
1,398721,1.312390,103.881504
2,629875,1.309135,103.679463
3,439731,1.305466,103.895674
4,659592,1.344619,103.749789
...,...,...,...
26234,389683,1.315271,103.892021
26235,538884,1.357136,103.890195
26236,828676,1.406937,103.896275
26237,556095,1.360203,103.869762


In [97]:
geocodes['postal'] = [int(item) if item.isdigit() else None for item in geocodes['postal']]

In [98]:
print(type(geocodes['postal'][0]))
print(type(moeschools2['postal'][0]))

<class 'numpy.float64'>
<class 'numpy.int64'>


In [99]:
moeschools_locations = moeschools2.merge(geocodes, how = 'left', on=['postal'])
moeschools_locations

,building,address,postal,mrt_desc,bus_desc,lat,lng
0,ADMIRALTY PRIMARY SCHOOL,11 WOODLANDS CIRCLE,738907,Admiralty Station,"TIBS 965, 964, 913",1.442550,103.800214
1,ADMIRALTY SECONDARY SCHOOL,31 WOODLANDS CRESCENT,737916,ADMIRALTY MRT,904,1.445891,103.802398
2,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11,768643,Yishun,Yishun Ring Road - 812 (white plate); Yishun A...,1.433414,103.832751
3,AHMAD IBRAHIM SECONDARY SCHOOL,751 YISHUN AVENUE 7,768928,"CANBERRA MRT, YISHUN MRT","117, 167, 169, 800, 811, 812, 883, 856, 858, 8...",1.436060,103.829719
4,AI TONG SCHOOL,100 Bright Hill Drive,579646,Bishan MRT,"410, 162, 52",1.360583,103.833020
...,...,...,...,...,...,...,...
341,ZHANGDE PRIMARY SCHOOL,51 Jalan Membina,169485,Tiong Bahru MRT Station,"16,33,63,123,195,605,851,14,65,147,167,196,197...",1.284212,103.825952
342,ZHENGHUA PRIMARY SCHOOL,9 Fajar Road,679002,LRT: Fajar Station,"190, 184, 187, 700, 700A, 921",1.379549,103.769313
343,ZHENGHUA SECONDARY SCHOOL,91 SENJA ROAD,677741,JELAPANG LRT,"920, 922, 976",1.388369,103.765522
344,ZHONGHUA PRIMARY SCHOOL,12 SERANGOON AVENUE 4,556095,"Ang Mo Kio MRT, Bishan MRT, Serangoon MRT","Boundary Road: 22,24,53,105,156,315,853,501,50...",1.360203,103.869762


In [100]:
missingschoolgeocode = moeschools_locations[moeschools_locations['lat'].isna()]
missingschoolgeocode
#missingschoolgeocode.to_excel('missingschoolgeocode.xlsx', index = False)

,building,address,postal,mrt_desc,bus_desc,lat,lng


In [101]:
moeschools_locations.to_csv('moeschools_locations.csv')

### NON MOE: PRIVATE / INTERNATIONAL / UNIVERSITIES / Higher Education Institutions (HEIs) and Private schools

In [102]:
intschools = pd.read_csv('intschool_mapper.csv')
intschools = intschools[['lat','lng','searchval','building','postal']]
intschools

,lat,lng,searchval,building,postal
0,1.306379,103.905164,5 STEPS ACADEMY,NIL,428833
1,1.301061,103.781310,ACS INDEPENDENT BOARDING SCHOOL,ACS INDEPENDENT BOARDING SCHOOL,139656
2,1.307747,103.798542,ACS INTERNATIONAL,ANGLO-CHINESE SCHOOL (INTERNATIONAL),278475
3,1.319923,103.834958,ACS OLDHAM HALL BOARDING SCHOOL,ACS OLDHAM HALL BOARDING SCHOOL,309937
4,1.301852,103.780141,ANGLO-CHINESE SCHOOL (INDEPENDENT),LIVING WATERS METHODIST CHURCH,139650
...,...,...,...,...,...
111,1.291595,103.857049,PSB ACADEMY,OCBC MARINA SQUARE,39594
112,1.300206,103.837317,RAFFLES COLLEGE OF HIGHER EDUCATION,111 SOMERSET,238164
113,1.274527,103.845777,SISH INSTITUTE,MARKETING INSTITUTE OF SINGAPORE,79904
114,1.275215,103.801330,SINGAPORE RAFFLES MUSIC COLLEGE,HUNDRED THOUGHTS PTE LTD,119962


In [75]:
'''
## Locations for Higher Education Institutions (HEIs) and Private schools
## Exclude government lower education schools and special education schools
nonMOE_locations = allschool_locations[~allschool_locations['building'].str.contains("SECONDARY SCHOOL|SKOOL|PRIMARY SCHOOL|AWWA SCHOOL|RAINBOW|SAINT PATRICK'S SCHOOL|AUTISM RESOURCE CENTRE|VICTORIA SCHOOL")]
## Exclude MOE schools that does not contain the filter words
nonMOE_locations = nonMOE_locations[~nonMOE_locations['postal'].isin(moeschools['postal_code'])] #|METHODIST GIRLS' SCHOOL|CANOSSIAN SCHOOL
nonMOE_locations = nonMOE_locations.reset_index(drop = True)
nonMOE_locations
'''

'\n## Locations for Higher Education Institutions (HEIs) and Private schools\n## Exclude government lower education schools and special education schools\nnonMOE_locations = allschool_locations[~allschool_locations[\'building\'].str.contains("SECONDARY SCHOOL|SKOOL|PRIMARY SCHOOL|AWWA SCHOOL|RAINBOW|SAINT PATRICK\'S SCHOOL|AUTISM RESOURCE CENTRE|VICTORIA SCHOOL")]\n## Exclude MOE schools that does not contain the filter words\nnonMOE_locations = nonMOE_locations[~nonMOE_locations[\'postal\'].isin(moeschools[\'postal_code\'])] #|METHODIST GIRLS\' SCHOOL|CANOSSIAN SCHOOL\nnonMOE_locations = nonMOE_locations.reset_index(drop = True)\nnonMOE_locations\n'

In [74]:
'''
### DATA INVESTIGATION
# Verify if moeschools dataset in Generalinformationofschools.csv matches with the data in sg_zipcode_mapper_utf.csv
moeschools[(moeschools['postal_code'] == 599986)]
'''

"\n### DATA INVESTIGATION\n# Verify if moeschools dataset in Generalinformationofschools.csv matches with the data in sg_zipcode_mapper_utf.csv\nmoeschools[(moeschools['postal_code'] == 599986)]\n"

In [77]:
'''
allschool_locations[(allschool_locations['building'] == "METHODIST GIRLS' SCHOOL")]
'''

'\nallschool_locations[(allschool_locations[\'building\'] == "METHODIST GIRLS\' SCHOOL")]\n'

In [103]:
intschools.to_csv('intschool_locations.csv')

## JOIN MOE SCHOOLS WITH ALL SCHOOLS

In [81]:
allschool_locations

,lat,lng,searchval,building,postal
0,1.301061,103.781310,ACS INDEPENDENT BOARDING SCHOOL,ACS INDEPENDENT BOARDING SCHOOL,139656
1,1.319923,103.834958,ACS OLDHAM HALL BOARDING SCHOOL,ACS OLDHAM HALL BOARDING SCHOOL,309937
2,1.445891,103.802398,ADMIRALTY SECONDARY SCHOOL,ADMIRALTY SECONDARY SCHOOL,737916
3,1.379838,103.845603,ANDERSON JUNIOR COLLEGE STUDENT HOSTEL,ANDERSON JUNIOR COLLEGE STUDENT HOSTEL,567736
4,1.331181,103.941853,ANGLICAN HIGH SCHOOL,ANGLICAN HIGH SCHOOL,487012
...,...,...,...,...,...
830,1.300206,103.837317,RAFFLES COLLEGE OF HIGHER EDUCATION,111 SOMERSET,238164
831,1.275215,103.801330,SINGAPORE RAFFLES MUSIC COLLEGE,HUNDRED THOUGHTS PTE LTD,119962
832,1.357136,103.890195,YUYING SECONDARY SCHOOL,YUYING SECONDARY SCHOOL,538884
833,1.406937,103.896275,YUSOF ISHAK SECONDARY SCHOOL,YUSOF ISHAK SECONDARY SCHOOL,828676


In [83]:
moeschools_locations = moeschools_locations[['lat','lng','building','postal']]
moeschools_locations

,lat,lng,building,postal
0,1.442550,103.800214,ADMIRALTY PRIMARY SCHOOL,738907
1,1.445891,103.802398,ADMIRALTY SECONDARY SCHOOL,737916
2,1.433414,103.832751,AHMAD IBRAHIM PRIMARY SCHOOL,768643
3,1.436060,103.829719,AHMAD IBRAHIM SECONDARY SCHOOL,768928
4,1.360583,103.833020,AI TONG SCHOOL,579646
...,...,...,...,...
341,1.284212,103.825952,ZHANGDE PRIMARY SCHOOL,169485
342,1.379549,103.769313,ZHENGHUA PRIMARY SCHOOL,679002
343,1.388369,103.765522,ZHENGHUA SECONDARY SCHOOL,677741
344,1.360203,103.869762,ZHONGHUA PRIMARY SCHOOL,556095


In [88]:
intschools = intschools[['lat','lng','searchval','postal']]
intschools = intschools.rename(columns={'searchval':'building'})
intschools

,lat,lng,building,postal
0,1.306379,103.905164,5 STEPS ACADEMY,428833
1,1.301061,103.781310,ACS INDEPENDENT BOARDING SCHOOL,139656
2,1.307747,103.798542,ACS INTERNATIONAL,278475
3,1.319923,103.834958,ACS OLDHAM HALL BOARDING SCHOOL,309937
4,1.301852,103.780141,ANGLO-CHINESE SCHOOL (INDEPENDENT),139650
...,...,...,...,...
111,1.291595,103.857049,PSB ACADEMY,39594
112,1.300206,103.837317,RAFFLES COLLEGE OF HIGHER EDUCATION,238164
113,1.274527,103.845777,SISH INSTITUTE,79904
114,1.275215,103.801330,SINGAPORE RAFFLES MUSIC COLLEGE,119962


In [89]:

all_school_locations = pd.concat([intschools, moeschools_locations], ignore_index=True)

all_school_locations

,lat,lng,building,postal
0,1.306379,103.905164,5 STEPS ACADEMY,428833
1,1.301061,103.781310,ACS INDEPENDENT BOARDING SCHOOL,139656
2,1.307747,103.798542,ACS INTERNATIONAL,278475
3,1.319923,103.834958,ACS OLDHAM HALL BOARDING SCHOOL,309937
4,1.301852,103.780141,ANGLO-CHINESE SCHOOL (INDEPENDENT),139650
...,...,...,...,...
457,1.284212,103.825952,ZHANGDE PRIMARY SCHOOL,169485
458,1.379549,103.769313,ZHENGHUA PRIMARY SCHOOL,679002
459,1.388369,103.765522,ZHENGHUA SECONDARY SCHOOL,677741
460,1.360203,103.869762,ZHONGHUA PRIMARY SCHOOL,556095


In [90]:
all_school_locations = all_school_locations.drop_duplicates()
all_school_locations

,lat,lng,building,postal
0,1.306379,103.905164,5 STEPS ACADEMY,428833
1,1.301061,103.781310,ACS INDEPENDENT BOARDING SCHOOL,139656
2,1.307747,103.798542,ACS INTERNATIONAL,278475
3,1.319923,103.834958,ACS OLDHAM HALL BOARDING SCHOOL,309937
4,1.301852,103.780141,ANGLO-CHINESE SCHOOL (INDEPENDENT),139650
...,...,...,...,...
457,1.284212,103.825952,ZHANGDE PRIMARY SCHOOL,169485
458,1.379549,103.769313,ZHENGHUA PRIMARY SCHOOL,679002
459,1.388369,103.765522,ZHENGHUA SECONDARY SCHOOL,677741
460,1.360203,103.869762,ZHONGHUA PRIMARY SCHOOL,556095


In [91]:
all_school_locations = all_school_locations.drop_duplicates(subset=['postal'], keep=False)
all_school_locations = all_school_locations.reset_index(drop = True)
all_school_locations

,lat,lng,building,postal
0,1.306379,103.905164,5 STEPS ACADEMY,428833
1,1.301061,103.781310,ACS INDEPENDENT BOARDING SCHOOL,139656
2,1.319923,103.834958,ACS OLDHAM HALL BOARDING SCHOOL,309937
3,1.311591,103.848500,APS SWIM SCHOOL,218253
4,1.296196,103.815637,ASTOR INTERNATIONAL SCHOOL,248922
...,...,...,...,...
440,1.284212,103.825952,ZHANGDE PRIMARY SCHOOL,169485
441,1.379549,103.769313,ZHENGHUA PRIMARY SCHOOL,679002
442,1.388369,103.765522,ZHENGHUA SECONDARY SCHOOL,677741
443,1.360203,103.869762,ZHONGHUA PRIMARY SCHOOL,556095


In [92]:
check = all_school_locations[all_school_locations.duplicated(subset=['postal'], keep=False)]
check

,lat,lng,building,postal


In [93]:
all_school_locations.to_csv('all_school_locations.csv')